# Implementing "Fundamental Analysis via Machine Learning" (Ceo et al. 2024)
This paper can be found at https://doi.org/10.1080/0015198X.2024.2313692

## 1. Imports

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

## 2 Data Preprocessing

### 2.1 Load dataset

In [2]:
# Compustat dataset downloaded from WRDS with only the necessary columns
df = pd.read_csv('datasets/compustat_crsp.csv')
 
# print fields 
print(df.columns)

Index(['gvkey', 'datadate', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic',
       'conm', 'curcd', 'act', 'ap', 'at', 'ceq', 'che', 'cogs', 'csho', 'dlc',
       'dltt', 'dp', 'dvc', 'ib', 'intan', 'invt', 'ivaeq', 'ivao', 'lct',
       'lt', 'nopi', 'oancf', 'ppent', 're', 'rect', 'sale', 'spi', 'txp',
       'txt', 'xad', 'xido', 'xidoc', 'xint', 'xrd', 'xsga', 'costat',
       'dvpsx_f', 'conml', 'sic', 'LINKTYPE', 'LPERMNO', 'PERMNO', 'MthCalDt',
       'MthPrc', 'MthRet', 'SecurityType', 'SecuritySubType', 'PrimaryExch',
       'ShrOut'],
      dtype='object')


### 2.2 Clean, Filter and Format the data
The below is from the paper
We further impose the following data requirements:
1. The following financial statement items must be non-missing: total assets, sales revenue, income before extraordinary items, and common shares outstanding
2. The stocks must be ordinary common shares listed on the NYSE, AMEX, or NASDAQ
3. The firms cannot be in the financial (SIC 6000–6999) or regulated utilities (SIC 4900–4999) industries
4. The stock prices at the end of the third month after the end of the fiscal year must be greater than US$1.

In [3]:
# 1) 
df = df.dropna(subset=['at', 'sale', 'ib', 'csho'])    # Filter out records with missing data in these columns

# 2)
df = df[(df['SecurityType'] == 'EQTY') & (df['SecuritySubType'] == 'COM')] # Common equity
df = df[df['PrimaryExch'].isin(['N', 'A', 'Q'])] # NYSE ('N'), AMEX ('A'), NASDAQ ('Q')

# 3)
df = df[~df['sic'].isin(range(6000, 7000))]  # Exclude financial firms
df = df[~df['sic'].isin(range(4900, 5000))]  # Exclude regulated utilities

# 4)
df = df[df['MthPrc'] > 1]

# Sort values based on GVKEY (unique firm identifier) and year 
df = df.sort_values(by=['gvkey', 'datadate'])

# Convert into time series
df['datadate'] = pd.to_datetime(df['datadate'])

### 2.3 Clean and format features
Creating new features as needed for the model. Created as per the formulas in the appendix of the paper

In [4]:
# Calculate iva (inventory and assets) as per the paper
# Add inventory and assets equties and inventory and assets other to get total inventory and assets
df['iva'] = df['ivaeq'].fillna(0) + df['ivao'].fillna(0)

# Calculate CFO (Cash flow from operations) as per the cited paper - NEED TO GO THROUGH
def calculate_cfo(df_in):
    """
    Calculates the final CFO feature by first calculating CFO from the
    balance sheet, then combining it with the reported CFO.
    
    Balance Sheet CFO = ib - Accruals
    Where Accruals = Δ(act - che) - Δ(lct - dlc - txp) - dp
    """
    df = df_in.copy()
    
    # 1. Calculate Balance Sheet CFO
    
    # Fill NAs with 0 for this calculation, as it's an imputation step
    ib = df['ib'].fillna(0)
    act = df['act'].fillna(0)
    che = df['che'].fillna(0)
    lct = df['lct'].fillna(0)
    dlc = df['dlc'].fillna(0)
    txp = df['txp'].fillna(0)
    dp = df['dp'].fillna(0)

    # Calculate non-cash current assets
    df['non_cash_ca'] = act - che
    
    # Calculate adjusted current liabilities 
    df['adj_cl'] = lct - dlc - txp
    
    # Calculate year-over-year changes (Δ)
    # Assumes df is already sorted by gvkey, datadate
    df['change_non_cash_ca'] = df.groupby('gvkey')['non_cash_ca'].diff()
    df['change_adj_cl'] = df.groupby('gvkey')['adj_cl'].diff()
    
    # Calculate Total Accruals
    df['total_accruals'] = (
        df['change_non_cash_ca'].fillna(0) - 
        df['change_adj_cl'].fillna(0) - 
        dp
    )
    
    # Calculate CFO from balance sheet
    df['cfo_bs'] = ib - df['total_accruals']
    
    # 2. Combine with Reported CFO
    
    # Calculate reported CFO
    df['cfo_reported'] = df['oancf'].fillna(0) - df['xidoc'].fillna(0)
    
    # Use reported CFO if available (and not 0), otherwise use balance sheet CFO
    df['cfo'] = df['cfo_reported'].replace(0, np.nan).fillna(df['cfo_bs'])
    
    # Return just the final 'cfo' column
    return df['cfo']

df['cfo'] = calculate_cfo(df)

### 2.4 Scale data
Scale by common shares outstanding to get per-share vaules. This means all vars will be comparable across companies. Careful about dividing by 0 errors.

In [5]:
# Define base features to be used in the model
base_features = [
        'ib', 'sale', 'cogs', 'xsga', 'dp', 'xint', 'nopi', 'txt', # From Category I
        'xad', 'xrd', 'spi', 'xido', 'dvc',                      # From Category II
        'at', 'act', 'lct', 'lt', 'ceq', 'che', 'invt', 'rect',  # From Category III
        'ppent', 'iva', 'intan', 'ap', 'dlc', 'txp', 'dltt', 're',
        'cfo'                                                    # From Category IV
]

# Fill missing values with 0
df[base_features] = df[base_features].fillna(0)

# Will store the names of the new per-share features
per_share_features = []

# Scaling
for col in base_features:
    col_per_share = f"{col}_per_share"
    df[col_per_share] = df[col] / df['csho']
    per_share_features.append(col_per_share)

# Handling infs
df = df.replace([np.inf, -np.inf], np.nan)

# Dropping rows with NaN values
df = df.dropna()

### 2.5 Target Variable
The target is one-year-forward earnings per share. The equation is below:
$$y_t = \frac{E_{t+1}}{csho_{t+1}}$$

In [6]:
# Get E_t+1 (next year's 'ib')
df['ib_t1'] = df.groupby('gvkey')['ib'].shift(-1)

# Get csho_t+1 (next year's 'csho')
df['csho_t1'] = df.groupby('gvkey')['csho'].shift(-1)

# Calculate the target variable
df['y'] = df['ib_t1'] / df['csho_t1']

### 2.6 Create difference features
Calculate year-over-year change for all 30 per-share features

In [7]:
# Will store the names of the new difference features
diff_features = []

# Group by gvkey so that the diff is computed for the same company
df_diffs = df.groupby('gvkey')[per_share_features].diff()

# Compute diffs
for col in per_share_features:
    diff_col = f"{col}_diff"
    df[diff_col] = df_diffs[col]
    diff_features.append(diff_col)

### 2.7 Final Cleanup

In [ ]:
X_cols = per_share_features + diff_features  # List of 60 features
y_col = 'y'

# Only keep required cols
# Drop intermediate cols used for computation
df = df[['gvkey', 'datadate'] + X_cols + [y_col]]

# Drop rows with any NaN values
df = df.dropna()

,gvkey,datadate,ib_per_share,sale_per_share,cogs_per_share,xsga_per_share,dp_per_share,xint_per_share,nopi_per_share,txt_per_share,...,ppent_per_share_diff,iva_per_share_diff,intan_per_share_diff,ap_per_share_diff,dlc_per_share_diff,txp_per_share_diff,dltt_per_share_diff,re_per_share_diff,cfo_per_share_diff,y
36,1004,1990-05-31,1.595262,27.662915,20.950815,3.352319,0.446524,0.621129,0.080400,0.777267,...,0.773504,0.054185,-0.030628,-0.022036,-2.286434,0.004510,2.891729,1.023601,1.198643,0.931408
37,1004,1991-05-31,0.931408,29.358882,23.034422,3.684161,0.519539,0.633881,0.064376,0.412183,...,0.045778,-0.027453,-0.008673,-0.019891,-1.064711,0.063879,-0.158390,0.470445,1.116988,0.630228
38,1004,1992-05-31,0.630228,26.583873,20.822442,3.579470,0.513303,0.525568,0.078370,0.226429,...,-0.190259,-0.012995,-0.013874,0.520745,0.534417,0.060380,-0.104706,0.176788,-1.767128,0.017798
39,1004,1993-05-31,0.017798,24.072700,19.334381,3.276083,0.434438,0.509842,0.053267,-0.138356,...,-0.275303,-0.037301,-0.020619,-0.685269,0.001060,-0.162734,-0.064994,-0.462733,0.502540,0.596253
40,1004,1994-05-31,0.596253,25.635232,20.656482,3.148875,0.457815,0.601283,0.089526,0.264051,...,-0.080889,0.150753,-0.016350,1.072788,-1.538091,0.115695,3.106385,0.010941,-0.635879,0.655535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332846,343180,2022-12-31,-0.707140,0.000000,0.000000,0.862195,0.003421,0.000000,0.156129,0.000000,...,1.432549,-0.053467,0.021209,-0.012584,0.023313,0.000000,0.079768,-0.491620,-0.103981,-0.514978
332857,345920,2021-12-31,0.300686,10.744991,8.398696,1.522816,0.334708,0.047918,-0.004572,-0.428908,...,1.480293,0.000000,10.082668,0.331859,0.079298,0.016339,3.077712,1.396818,0.327996,-6.314910
332858,345920,2022-12-31,-6.314910,7.622209,6.556165,1.758480,0.918800,0.242450,0.018607,-0.142554,...,0.430114,0.000000,-5.001768,-0.705985,0.033768,-0.006360,0.327371,-6.403669,1.496578,-1.415440
332862,345980,2021-12-31,-0.548632,3.168693,1.471125,2.232523,0.013678,0.000000,0.024316,0.015198,...,-0.062652,0.019022,0.000000,-0.637529,-0.010172,0.000000,-0.040420,-0.144312,-0.196566,-13.083495
